In [ ]:
%run "../includes/configuration"
%run "../includes/common_functions"

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType


In [ ]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)

])

In [ ]:
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)
])

In [ ]:
drivers_df = spark.read \
.schema(drivers_schema) \
.json(f"{raw_folder_path}/drivers.json")

In [ ]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [ ]:
drivers_with_columns_df = drivers_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("ingestion_date", current_timestamp()) \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [ ]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

In [ ]:
drivers_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/drivers")